In [77]:
import os
import time
import math
import ephem
from astropy.time import Time
from astropy import units as u
import numpy as np
import networkx as nx


gen_data = 'gen_data/'

# WGS72 value; taken from https://geographiclib.sourceforge.io/html/NET/NETGeographicLib_8h_source.html
EARTH_RADIUS = 6378135.0

# GENERATION CONSTANTS   Iridium

BASE_NAME = "Lion"
NICE_NAME = "Lion"

# Lion 630

ECCENTRICITY = 0.0000001  # Circular orbits are zero, but pyephem does not permit 0, so lowest possible value
ARG_OF_PERIGEE_DEGREE = 0.0
PHASE_DIFF = True

################################################################
# The below constants are taken from Kuiper's FCC filing as below:
# [1]: https://www.itu.int/ITU-R/space/asreceived/Publication/DisplayPublication/8716
################################################################

MEAN_MOTION_REV_PER_DAY = 14.80  # Altitude ~630 km
ALTITUDE_M = 630000  # Altitude ~630 km

# Considering an elevation angle of 30 degrees; possible values [1]: 20(min)/30/35/45
SATELLITE_CONE_RADIUS_M = ALTITUDE_M / math.tan(math.radians(20.0))

MAX_GSL_LENGTH_M = math.sqrt(math.pow(SATELLITE_CONE_RADIUS_M, 2) + math.pow(ALTITUDE_M, 2))

# ISLs are not allowed to dip below 80 km altitude in order to avoid weather conditions
# MAX_ISL_LENGTH_M = 2 * math.sqrt(math.pow(EARTH_RADIUS + ALTITUDE_M, 2) - math.pow(EARTH_RADIUS + 80000, 2))
MAX_ISL_LENGTH_M = 8000000

light_speed = 3e8

simulation_end_time_s = 600
time_step_s = 100



class Mytopo():
    def __init__(self, enable_verbose_logs = False, **opts):
        super(Mytopo, self).__init__()

            # Graphs
        graphs_sat_net_graph_only_satellites_with_isls = nx.Graph()
        graphs_sat_net_graph_all_with_only_gsls = nx.Graph()
#####################################################################
        sat_net_switches_all_with_only_gsls = {}

        sat_info = read_tles(gen_data)
        # Dictionary:{
        # "n_orbits": n_orbits,
        # "n_sats_per_orbit": n_sats_per_orbit,
        # "num_of_all_satellite": n_orbits * n_sats_per_orbit,
        # "epoch": epoch,
        # "satellites":satellites
        # }
        ground_stations = read_ground_stations_extended(gen_data)
        satellites = sat_info['satellites']
        epoch = sat_info['epoch']
        time = epoch + 0 * u.day

        # graph Information
        for i in range(len(satellites)):
            graphs_sat_net_graph_only_satellites_with_isls.add_node(i)
        for i in range(len(satellites) + len(ground_stations)):
            graphs_sat_net_graph_all_with_only_gsls.add_node(i)

        # mininet network info
        # for i in range(len(satellites) + len(ground_stations)):
        #     sat_net_switches_all_with_only_gsls[i] = self.addSwitch(i)


        isl_list = read_isls(gen_data, sat_info['num_of_all_satellite'])

        total_num_isls = 0
        num_isls_per_sat = [0] * len(satellites)
        sat_neighbor_to_if = {}
        for (a, b) in isl_list:
            sat_distance_m = distance_m_between_satellites(satellites[a], satellites[b], str(epoch), str(time))
            if sat_distance_m > MAX_ISL_LENGTH_M:
                raise ValueError(
                    "The distance between two satellites (%d and %d) "
                    "with an ISL exceeded the maximum ISL length (%.2fm > %.2fm at t=%dns)"
                    % (a, b, sat_distance_m, MAX_ISL_LENGTH_M, time)
                )

            delay =round(sat_distance_m/light_speed, 3) * 1000

            # Add to network graph
            #self.addLink(sat_net_switches_all_with_only_gsls[a], sat_net_switches_all_with_only_gsls[b], bw = 10, delay = str(delay))
            graphs_sat_net_graph_only_satellites_with_isls.add_edge(a, b, weight=sat_distance_m)


            # Interface mapping of ISLs
            sat_neighbor_to_if[(a, b)] = num_isls_per_sat[a]
            sat_neighbor_to_if[(b, a)] = num_isls_per_sat[b]
            num_isls_per_sat[a] += 1
            num_isls_per_sat[b] += 1
            total_num_isls += 1


        if enable_verbose_logs:
            print("  > Total ISLs............. " + str(len(isl_list)))
            print("  > Min. ISLs/satellite.... " + str(np.min(num_isls_per_sat)))
            print("  > Max. ISLs/satellite.... " + str(np.max(num_isls_per_sat)))

        #################################

        # Calculate shortest path distances
        if enable_verbose_logs:
            print("  > Calculating Floyd-Warshall for graph without ground-station relays")
        # (Note: Numpy has a deprecation warning here because of how networkx uses matrices)
        dist_sat_net_without_gs = nx.floyd_warshall_numpy(graphs_sat_net_graph_only_satellites_with_isls)

        for time_since_epoch_s in range(0, simulation_end_time_s, time_step_s):

            time = epoch + time_since_epoch_s * u.s
            if enable_verbose_logs:
                print('\n\n')
                print("  > Epoch.................. " + str(epoch))
                print("  > Time since epoch....... " + str(time_since_epoch_s) + " s")
                print("  > Absolute time.......... " + str(time))

            ground_station_satellites_in_range = []
            for ground_station in ground_stations:
                # Find satellites in range
                satellites_in_range = []
                for sid in range(len(satellites)):
                    distance_m = distance_m_ground_station_to_satellite(
                        ground_station,
                        satellites[sid],
                        str(epoch),
                        str(time)
                    )
                    if distance_m <= MAX_GSL_LENGTH_M:
                        satellites_in_range.append((distance_m, sid))
                        #graph info
                        graphs_sat_net_graph_all_with_only_gsls.add_edge(sid, len(satellites) + ground_station["gid"], weight=distance_m)

                satellites_in_range = sorted(satellites_in_range)

                ground_station_satellites_in_range.append(satellites_in_range)

            if enable_verbose_logs:
                print('-------------------------------------')
                print(" ----> ground_station_satellites_in_range ")
                print(ground_station_satellites_in_range)


            ground_fstate_dict = calculate_fstate_shortest_path_without_gs_relaying(
            satellites,
            ground_stations,
            dist_sat_net_without_gs,
            ground_station_satellites_in_range,
            enable_verbose_logs = False)

            if enable_verbose_logs:
                print('  >fstate infomations.....   ')
                print(ground_fstate_dict)

            # add GSL link
            for src_ground_id, des_ground_id in ground_fstate_dict:

                adj_satellite_id = ground_fstate_dict.get((src_ground_id, des_ground_id))
                gsl_distance_m = distance_m_ground_station_to_satellite(
                        ground_stations[src_ground_id - len(satellites)],
                        satellites[adj_satellite_id],
                        str(epoch),
                        str(time)
                    )
                gsl_delay = round(gsl_distance_m/light_speed, 3) * 1000
                #self.addLink(sat_net_graph_all_with_only_gsls[src_ground_id], sat_net_graph_all_with_only_gsls[adj_satellite_id], loss = 0, delay = gsl_delay)

        # add ground host
        Hosts_On_Ground = {}
        for i in range(len(ground_stations)):
            Hosts_On_Ground[i] = self.addHOst(i, ip = '10.0.{}.{}'.format((len(satellites) + i + 1)//255,(i + 1+ len(satellites))%255))
            ground_id = len(satellites) + i
            self.addLink(Hosts_On_Ground[i], sat_net_switches_all_with_only_gsls[ground_id])


def read_tles(gen_data):
    """
    Read a constellation of satellites from the TLES file.

    :param filename_tles:                    Filename of the TLES (typically /path/to/tles.txt)

    :return: Dictionary:{
        "n_orbits": n_orbits,
        "n_sats_per_orbit": n_sats_per_orbit,
        "num_of_all_satellite": n_orbits * n_sats_per_orbit,
        "epoch": epoch,
        "satellites":satellites
        }
    """

    sat_data = os.listdir(gen_data)
    satellites = []
    with open(gen_data + sat_data[0] +'/tles.txt', 'r') as f:
        n_orbits, n_sats_per_orbit = [int(n) for n in f.readline().split()]
        universal_epoch = None

        i = 0
        for tles_line_1 in f:
            tles_line_2 = f.readline()
            tles_line_3 = f.readline()

            # Retrieve name and identifier
            name = tles_line_1
            sid = int(name.split()[1])
            if sid != i:
                raise ValueError("Satellite identifier is not increasing by one each line")
            i += 1

            # Fetch and check the epoch from the TLES data
            # In the TLE, the epoch is given with a Julian data of yyddd.fraction
            # ddd is actually one-based, meaning e.g. 18001 is 1st of January, or 2018-01-01 00:00.
            # As such, to convert it to Astropy Time, we add (ddd - 1) days to it
            # See also: https://www.celestrak.com/columns/v04n03/#FAQ04
            epoch_year = tles_line_2[18:20]
            epoch_day = float(tles_line_2[20:32])
            epoch = Time("20" + epoch_year + "-01-01 00:00:00", scale="tdb") + (epoch_day - 1) * u.day
            if universal_epoch is None:
                universal_epoch = epoch
            if epoch != universal_epoch:
                raise ValueError("The epoch of all TLES must be the same")

            # Finally, store the satellite information
            satellites.append(ephem.readtle(tles_line_1, tles_line_2, tles_line_3))

    return {
        "n_orbits": n_orbits,
        "n_sats_per_orbit": n_sats_per_orbit,
        "num_of_all_satellite": n_orbits * n_sats_per_orbit,
        "epoch": epoch,
        "satellites":satellites
    }

def read_isls(gen_filename, num_satellites):
    """
    Read ISLs file into a list of undirected edges

    :param filename_isls:  Filename of ISLs (typically /path/to/isls.txt)
    :param num_satellites: Number of satellites (to verify indices)

    :return: List of all undirected ISL edges
    """
    isls_list = []

    info = os.listdir(gen_filename)

    with open(gen_filename + info[0] + '/isls.txt', 'r') as f:
        isls_set = set()
        for line in f:
            line_spl = line.split()
            a = int(line_spl[0])
            b = int(line_spl[1])

            # Verify the input
            if a >= num_satellites:
                raise ValueError("Satellite does not exist: %d" % a)
            if b >= num_satellites:
                raise ValueError("Satellite does not exist: %d" % b)
            if b <= a:
                raise ValueError("The second satellite index must be strictly larger than the first")
            if (a, b) in isls_set:
                raise ValueError("Duplicate ISL: (%d, %d)" % (a, b))
            isls_set.add((a, b))

            # Finally add it to the list
            isls_list.append((a, b))

    return isls_list

def distance_m_between_satellites(sat1, sat2, epoch_str, date_str):
    """
    Computes the straight distance between two satellites in meters.

    :param sat1:       The first satellite
    :param sat2:       The other satellite
    :param epoch_str:  Epoch time of the observer (string)
    :param date_str:   The time instant when the distance should be measured (string)

    :return: The distance between the satellites in meters
    """

    # Create an observer somewhere on the planet
    observer = ephem.Observer()
    observer.epoch = epoch_str
    observer.date = date_str
    observer.lat = 0
    observer.lon = 0
    observer.elevation = 0

    # Calculate the relative location of the satellites to this observer
    sat1.compute(observer)
    sat2.compute(observer)

    # Calculate the angle observed by the observer to the satellites (this is done because the .compute() calls earlier)
    angle_radians = float(repr(ephem.separation(sat1, sat2)))

    # Now we have a triangle with three knows:
    # (1) a = sat1.range (distance observer to satellite 1)
    # (2) b = sat2.range (distance observer to satellite 2)
    # (3) C = angle (the angle at the observer point within the triangle)
    #
    # Using the formula:
    # c^2 = a^2 + b^2 - 2 * a * b * cos(C)
    #
    # This gives us side c, the distance between the two satellites
    return math.sqrt(sat1.range ** 2 + sat2.range ** 2 - (2 * sat1.range * sat2.range * math.cos(angle_radians)))

def distance_m_ground_station_to_satellite(ground_station, satellite, epoch_str, date_str):
    """
    Computes the straight distance between a ground station and a satellite in meters

    :param ground_station:  The ground station
    :param satellite:       The satellite
    :param epoch_str:       Epoch time of the observer (ground station) (string)
    :param date_str:        The time instant when the distance should be measured (string)

    :return: The distance between the ground station and the satellite in meters
    """

    # Create an observer on the planet where the ground station is
    observer = ephem.Observer()
    observer.epoch = epoch_str
    observer.date = date_str
    observer.lat = str(ground_station["latitude_degrees_str"])   # Very important: string argument is in degrees.
    observer.lon = str(ground_station["longitude_degrees_str"])  # DO NOT pass a float as it is interpreted as radians
    observer.elevation = ground_station["elevation_m_float"]

    # Compute distance from satellite to observer
    satellite.compute(observer)

    # Return distance
    return satellite.range

def read_ground_stations_extended(gen_filename):
    """
    Reads ground stations from the input file.

    :param filename_ground_stations_basic: Filename of ground stations basic (typically /path/to/ground_stations.txt)

    :return: List of ground stations
    """

    constellation_info_list = os.listdir(gen_filename)

    ground_stations_extended = []
    gid = 0

    for i in constellation_info_list:
        with open(gen_filename + i +'/ground_stations.txt', 'r') as f:
            for line in f:
                split = line.split(',')
                if len(split) != 8:
                    raise ValueError("Extended ground station file has 8 columns: " + line)
                if int(split[0]) != gid:
                    raise ValueError("Ground station id must increment each line")
                ground_station_basic = {
                    "gid": gid,
                    "name": split[1],
                    "latitude_degrees_str": split[2],
                    "longitude_degrees_str": split[3],
                    "elevation_m_float": float(split[4]),
                    "cartesian_x": float(split[5]),
                    "cartesian_y": float(split[6]),
                    "cartesian_z": float(split[7]),
                }
                ground_stations_extended.append(ground_station_basic)
                gid += 1
    return ground_stations_extended

def calculate_fstate_shortest_path_without_gs_relaying(
        satellites,
        ground_stations,
        dist_sat_net_without_gs,
        ground_station_satellites_in_range,
        enable_verbose_logs = False
):

    # Forwarding state
    fstate = {}
    # Satellites to ground stations
    # From the satellites attached to the destination ground station,
    # select the one which promises the shortest path to the destination ground station (getting there + last hop)
    dist_satellite_to_ground_station = {}
    for curr in range(len(satellites)):
        for dst_gid in range(len(ground_stations)):
            dst_gs_node_id = len(satellites) + dst_gid

            # Among the satellites in range of the destination ground station,
            # find the one which promises the shortest distance
            possible_dst_sats = ground_station_satellites_in_range[dst_gid]
            possibilities = []
            for b in possible_dst_sats:
                if not math.isinf(dist_sat_net_without_gs[(curr, b[1])]):  # Must be reachable
                    possibilities.append(
                        (
                            dist_sat_net_without_gs[(curr, b[1])] + b[0],
                            b[1]
                        )
                    )
            possibilities = list(sorted(possibilities))

            # By default, if there is no satellite in range for the

            distance_to_ground_station_m = float("inf")
            if len(possibilities) > 0:
                dst_sat = possibilities[0][1]
                distance_to_ground_station_m = possibilities[0][0]
            # In any case, save the distance of the satellite to the ground station to re-use
            # when we calculate ground station to ground station forwarding
            dist_satellite_to_ground_station[(curr, dst_gs_node_id)] = distance_to_ground_station_m

        # Ground stations to ground stations
        # Choose the source satellite which promises the shortest path
    for src_gid in range(len(ground_stations)):
        for dst_gid in range(len(ground_stations)):
            if src_gid != dst_gid:
                src_gs_node_id = len(satellites) + src_gid
                dst_gs_node_id = len(satellites) + dst_gid

                # Among the satellites in range of the source ground station,
                # find the one which promises the shortest distance
                possible_src_sats = ground_station_satellites_in_range[src_gid]

                if enable_verbose_logs:
                    print('-----------------------')
                    print('----possible_src_sats----')
                    print(possible_src_sats)

                possibilities = []
                for a in possible_src_sats:
                    best_distance_offered_m = dist_satellite_to_ground_station[(a[1], dst_gs_node_id)]
                    if not math.isinf(best_distance_offered_m):
                        possibilities.append(
                            (
                                a[0] + best_distance_offered_m,  # distance between two stations #(51, 100): 725104.4375, (51, 101): 15010708.41926724, the two value is addde
                                a[1]
                            )
                        )
                possibilities = sorted(possibilities)

                if enable_verbose_logs:
                    print('-------------------------------')
                    print("dist_satellite_to_ground_station")
                    print(possibilities)


                # By default, if there is no satellite in range for one of the
                # ground stations, it will be dropped (indicated by -1)
                #next_hop_decision = (-1, -1, -1)
                if len(possibilities) > 0:
                    src_sat_id = possibilities[0][1]
                else:
                    src_sat_id = -1

                fstate[(src_gs_node_id, dst_gs_node_id)] = src_sat_id

    return fstate


if __name__ == "__main__":
    test_topo = Mytopo(enable_verbose_logs = True)



  > Total ISLs............. 200
  > Min. ISLs/satellite.... 4
  > Max. ISLs/satellite.... 4
  > Calculating Floyd-Warshall for graph without ground-station relays



  > Epoch.................. 2000-01-01 00:00:00.000
  > Time since epoch....... 0 s
  > Absolute time.......... 2000-01-01 00:00:00.000
-------------------------------------
 ----> ground_station_satellites_in_range 
[[(725104.4375, 51), (1463304.875, 42)], [(823043.375, 73), (1653661.25, 1)]]
  >fstate infomations.....   
{(100, 101): 51, (101, 100): 73}



  > Epoch.................. 2000-01-01 00:00:00.000
  > Time since epoch....... 100 s
  > Absolute time.......... 2000-01-01 00:01:40.000
-------------------------------------
 ----> ground_station_satellites_in_range 
[[(1162528.375, 51), (1394945.75, 42), (1664749.25, 32)], [(1222338.5, 1), (1346829.5, 73), (1499032.375, 91)]]
  >fstate infomations.....   
{(100, 101): 51, (101, 100): 73}



  > Epoch.................. 2000-01-01 00:00:00.000
  > Time since epoch....

In [61]:

test_list = []

test_list.append(
    (3333,4))
test_list.append(
    (4555,3) )
test_list.append(
    (35,4)
)
test_list = sorted(test_list)
print(test_list)

print(test_list[0][0])



print(os.path.join('sss/','hhh', 'eee'))


constellations_info = os.listdir(gen_data)
for constellation in constellations_info:
    if os.path.isdir(os.path.join(gen_data,constellation)):
        each_info = os.listdir(os.path.join(gen_data,constellation))
        for info in each_info:
            if os.path.isdir(os.path.join(gen_data, constellation,info)):
                fstate_info = os.listdir(os.path.join(gen_data, constellation, info))
                fstate_info = sorted(fstate_info)
                print(fstate_info)
                for i in range(len(fstate_info)//2):

                    with open(os.path.join(gen_data, constellation, info, fstate_info[i]), 'r') as f:
                        print(f.name)

fstate = {}

fstate[(2, 4)] = 5
fstate[(2, 6)] = 7
print(fstate)
for i in fstate:
    print(fstate.get(i))

for a, b in fstate:
    print(fstate.get((a,b)))





[(35, 4), (3333, 4), (4555, 3)]
35
sss/hhh/eee
['fstate_0.txt', 'fstate_100000000000.txt', 'fstate_200000000000.txt', 'fstate_300000000000.txt', 'fstate_400000000000.txt', 'fstate_500000000000.txt', 'gsl_if_bandwidth_0.txt', 'gsl_if_bandwidth_100000000000.txt', 'gsl_if_bandwidth_200000000000.txt', 'gsl_if_bandwidth_300000000000.txt', 'gsl_if_bandwidth_400000000000.txt', 'gsl_if_bandwidth_500000000000.txt']
gen_data/Lion_isls_plus_grid_twostation_algorithm_free_one_only_over_isls/dynamic_state_100000ms_for_600s/fstate_0.txt
gen_data/Lion_isls_plus_grid_twostation_algorithm_free_one_only_over_isls/dynamic_state_100000ms_for_600s/fstate_100000000000.txt
gen_data/Lion_isls_plus_grid_twostation_algorithm_free_one_only_over_isls/dynamic_state_100000ms_for_600s/fstate_200000000000.txt
gen_data/Lion_isls_plus_grid_twostation_algorithm_free_one_only_over_isls/dynamic_state_100000ms_for_600s/fstate_300000000000.txt
gen_data/Lion_isls_plus_grid_twostation_algorithm_free_one_only_over_isls/dynamic

In [83]:
# add flowentry to switches by restful api

print(4%4)


0
